In [4]:

import pandas as pd
from pathlib import Path
import json
dataset_path = Path("/mnt/i/bio/mave_curation/Findlay_VHL_SGE/")
assert dataset_path.exists()
data = pd.read_excel(dataset_path/ "raw" / "41588_2024_1800_MOESM4_ESM.xlsx",header=2)

In [5]:
data

,hg38_pos,ref,alt,cHGVS,pHGVS,function_score_final,q_value,rna_score,rna_score_day_20,delta_rna,...,SpliceAI.acc.loss,SpliceAI.don.gain,SpliceAI.don.loss,max_spliceAI,ddG.VHL.complex,ddG.VHL.only,exposure,low_rna,mis_vep_category,novel.clinvar
0,10141958,G,C,c.111G>C,p.E37D,0.227104,0.999801,0.146014,0.049020,-0.096993,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
1,10141958,G,T,c.111G>T,p.E37D,0.480106,0.999801,0.496812,0.314450,-0.182362,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
2,10141959,T,A,c.112T>A,p.S38T,-0.065021,0.736202,-0.072035,-0.281796,-0.209761,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
3,10141959,T,G,c.112T>G,p.S38A,0.235124,0.999801,-2.064347,-1.778039,0.286308,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
4,10141960,C,A,c.113C>A,p.S38Y,-0.324690,0.124480,0.548857,0.549808,0.000951,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,10141956,G,T,c.109G>T,p.E37*,1.197395,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2264,10141947,G,C,c.100G>C,p.G34R,1.199658,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2265,10141951,C,A,c.104C>A,p.A35D,1.452510,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2266,10141936,G,T,c.89G>T,p.G30V,1.506749,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
from Bio.PDB.Polypeptide import protein_letters_3to1
protein_letters_1to3 = {v: k.title() for k, v in protein_letters_3to1.items()}
protein_letters_1to3['*'] = "Ter"
def make_hgvs_pro(s):
    return f"p.{protein_letters_1to3[s[2]]}{s[3:-1]}{protein_letters_1to3[s[-1]]}"

In [22]:
data = data.dropna(subset=['pHGVS',])
data = data.assign(hgvs_pro=data.pHGVS.apply(make_hgvs_pro)).rename(columns=dict(function_score_final='score'))

In [23]:
data

,hg38_pos,ref,alt,cHGVS,pHGVS,score,q_value,rna_score,rna_score_day_20,delta_rna,...,SpliceAI.don.gain,SpliceAI.don.loss,max_spliceAI,ddG.VHL.complex,ddG.VHL.only,exposure,low_rna,mis_vep_category,novel.clinvar,hgvs_pro
0,10141958,G,C,c.111G>C,p.E37D,0.227104,0.999801,0.146014,0.049020,-0.096993,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Glu37Asp
1,10141958,G,T,c.111G>T,p.E37D,0.480106,0.999801,0.496812,0.314450,-0.182362,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Glu37Asp
2,10141959,T,A,c.112T>A,p.S38T,-0.065021,0.736202,-0.072035,-0.281796,-0.209761,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Ser38Thr
3,10141959,T,G,c.112T>G,p.S38A,0.235124,0.999801,-2.064347,-1.778039,0.286308,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Ser38Ala
4,10141960,C,A,c.113C>A,p.S38Y,-0.324690,0.124480,0.548857,0.549808,0.000951,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Ser38Tyr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,10141956,G,T,c.109G>T,p.E37*,1.197395,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.Glu37Ter
2264,10141947,G,C,c.100G>C,p.G34R,1.199658,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.Gly34Arg
2265,10141951,C,A,c.104C>A,p.A35D,1.452510,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.Ala35Asp
2266,10141936,G,T,c.89G>T,p.G30V,1.506749,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.Gly30Val


In [24]:
scoreset = pd.DataFrame(data.groupby('hgvs_pro').score.mean()).reset_index()

In [25]:
scoreset

,hgvs_pro,score
0,p.Ala11Pro,0.018568
1,p.Ala122Ala,-0.098126
2,p.Ala122Glu,-0.123239
3,p.Ala122Gly,-0.459489
4,p.Ala122Pro,-0.815256
...,...,...
1269,p.Val87Glu,-0.626087
1270,p.Val87Gly,-2.599586
1271,p.Val87Ile,-0.078465
1272,p.Val87Leu,0.014823


In [26]:
scoreset.to_csv(dataset_path / "scoreset.csv", index=False)

In [27]:
metadata = dict(uniprot_acc='P40337')
with open(dataset_path / "metadata.json", "w") as f:
    json.dump(metadata, f)